In [127]:
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random

from icecream import ic

In [128]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES

,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000
...,...,...,...
721,Ziyang,30.140000,104.640000
722,Zoucheng,35.400000,116.966667
723,Zouxian,35.410000,116.940000
724,Zunhua,40.183333,117.966667


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [ ]:
def tsp_cost(tsp):
    # assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

# EA

## Parameters
These are the parameters we can play with. I found that increasing the tournament size leads to a faster convergence, alowing for a smaller population size and fewer generations.

In [130]:
POPULATION_SIZE = 1000
NUM_GENERATIONS = 10000
MUTATION_RATE = 0.1
NUM_CITIES = len(CITIES)
TOURNAMENT_SIZE = 5
ELITISM_RATE = 0.1

## Defining a fitness function
In this case we define the fitness as the total distance of the path

In [116]:
def fitness(path):
    return tsp_cost(path)

In [117]:
def fitness_improved(path):
    total_distance = 0
    num_cities = len(path)
    for i in range(num_cities):
        start_city = path[i]
        end_city = path[(i + 1) % num_cities]
        total_distance += DIST_MATRIX[start_city][end_city]
    return total_distance

## Population Initialization
We are generating an initial population by creating random permutations of cities

In [118]:
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        path = list(range(NUM_CITIES))
        random.shuffle(path)
        population.append(path)
    return population

## Parent Selection
Here we are using a pseudo uniform selection, where we select parents with a probability inversely proportional to their fitness.

In [ ]:
def selection(population, fitness_scores, num_parents):
    selected = random.choices(population, weights=[1/f for f in fitness_scores], k=num_parents)
    return selected

### Tournament Selection
We are implementing a tournament selection algorithm, where we select the best individual from a random subset of the population.

In [119]:
def tournament_selection(population, fitness_scores):
    selected = []
    for _ in range(len(population) // 2):
        participants = random.sample(list(zip(population, fitness_scores)), TOURNAMENT_SIZE)
        best = min(participants, key=lambda x: x[1])
        selected.append(best[0])
    return selected

## Crossover
We are implementing the ordered crossover operator

In [120]:
def xover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]

    pointer = end
    for city in parent2:
        if city not in child:
            if pointer >= len(child):
                pointer = 0
            child[pointer] = city
            pointer += 1
    return child

### Partially Mapped Crossover (PMX)
PMX preserves subsequences from each parent while maintaining city order, which tends to produce better offspring.

In [121]:
def pmx_xover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [-1] * size
    child[start:end] = parent1[start:end]

    for i in range(start, end):
        if parent2[i] not in child:
            value = parent2[i]
            pos = i
            while child[pos] != -1:
                pos = parent2.index(parent1[pos])
            child[pos] = value

    for i in range(size):
        if child[i] == -1:
            child[i] = parent2[i]

    return child

### Mutation
We are implementing the swap mutation operator

In [ ]:
def mutate(path):
    for i in range(len(path)):
        if random.random() < MUTATION_RATE:
            j = random.randint(0, len(path) - 1)
            path[i], path[j] = path[j], path[i]
    return path

### Inversion Mutation
This mutation takes a subsequence and reverses it, maintaining order but altering paths slightly

In [122]:
def inversion_mutation(path):
    if random.random() < MUTATION_RATE:
        start, end = sorted(random.sample(range(len(path)), 2))
        path[start:end] = reversed(path[start:end])
    return path

## Putting it all together
The first implementation is the most basic EA, without elitism

In [123]:
def evolutionary_algorithm():
    population = initialize_population(POPULATION_SIZE)
    best_path = None
    best_distance = float('inf')

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [fitness(path) for path in population]

        # Track the best solution
        min_distance = min(fitness_scores)
        ic(generation, min_distance)
        if min_distance < best_distance:
            best_distance = min_distance
            best_path = population[fitness_scores.index(min_distance)]

        # Selection
        parents = selection(population, fitness_scores, POPULATION_SIZE // 2)

        # Crossover and Mutation
        offspring = []
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i + 1]
            
            # We create two children from each pair of parents, to maintain the population size
            child1, child2 = xover(parent1, parent2), xover(parent2, parent1)
            offspring.append(mutate(child1))
            offspring.append(mutate(child2))

        population = offspring  # Replace old population with offspring

    return best_path, best_distance


### Elitism EA
Elitism helps by carrying the best paths directly into the next generation, ensuring progress.

In [131]:
def evolutionary_algorithm2():
    population = initialize_population(POPULATION_SIZE)
    best_path = None
    best_distance = float('inf')
    elite_size = int(ELITISM_RATE * POPULATION_SIZE)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [fitness_improved(path) for path in population]

        # Track the best solution
        min_distance = min(fitness_scores)
        # ic(generation, min_distance)
        if min_distance < best_distance:
            best_distance = min_distance
            best_path = population[fitness_scores.index(min_distance)]

        # Elitism: Keep the top elite_size paths
        sorted_population = [path for _, path in sorted(zip(fitness_scores, population))]  # sort by fitness
        elites = sorted_population[:elite_size]

        # Selection
        parents = tournament_selection(population, fitness_scores)

        # Crossover and Mutation
        offspring = []
        for i in range(0, len(parents) - 1, 2):
            parent1, parent2 = parents[i], parents[i + 1]
            child1 = pmx_xover(parent1, parent2)
            child2 = pmx_xover(parent2, parent1)
            offspring.append(inversion_mutation(child1))
            offspring.append(inversion_mutation(child2))

        # New population: elites + offspring
        population = elites + offspring[:POPULATION_SIZE - elite_size]

    return best_path, best_distance


In [132]:
bp, bd = evolutionary_algorithm2()

In [133]:
for i in bp:
    print(CITIES.iloc[i]['name'])
    
print(f'Total distance: {bd}')

Yingcheng
Xiaogan
Suizhou
Anlu
Lujiang
Chizhou
Jingdezhen
Anqing
Quzhou
Fuyang
Linhai
Yueqing
Wenling
Luqiao
Jiaojiang
Wenzhou
Putian
Shishi
Xiamen
Zengcheng
Nanxiong
Hepo
Shanwei
Xingning
Meizhou
Quanzhou
Qingyang
Nanping
Jian'ou
Ruian
Huangyan
Fuding
Changle
Bantou
Yongan
Sanming
Shaowu
Wuyishan
Shangrao
Leping
Guixi
Dexing
Hejin
Jiangshan
Jianyang
Zhangping
Longyan
Zhangzhou
Longhai
Chenghai
Huanggang
Anbu
Shantou
Mianchang
Haimen
Puning
Jieyang
Chaozhou
Haicheng
Luoyang
Huizhou
Shilong
Huicheng
Lufeng
Jiazi
Jieshi
Pingshan
Jiangmen
Shaping
Kaiping
Sihui
Gaoyao
Yangjiang
Taishan
Taicheng
Foshan
Huangpu
Shiqiao
Zhongshan
Zhuhai
Shenzhen
Danshui
Dongguan
Humen
Xiaolan
Daliang
Guangzhou
Lecheng
Chenzhou
Qidong
Hongjiang
Shaoyang
Zhuzhou
Miluo
Chibi
Linxiang
Changde
Zhangjiajie
Lichuan
Zunyi
Guiyang
Kaili
Bijie
Chishui
Renhuai
Fuling
Daxian
Wanyuan
Guangyuan
Dazhou
Huaying
Wanxian
Shangluo
Lingbao
Lüliang
Linfen
Jiexiu
Gujiao
Huozhou
Yulin
Hancheng
Yongji
Fenyang
Sanmenxia
Yuncheng
Houm